# Step 4 – Dimensional Model (Gold)

Creates a star schema from `stg_registrations_clean`.

Dimensions:
- dim_date
- dim_state
- dim_manufacturer
- dim_model

Fact:
- fct_registrations


2) Connect to DuckDB

In [1]:
import duckdb
from pathlib import Path

DB_PATH = Path("../data/duckdb/motorcycle.db")
con = duckdb.connect(str(DB_PATH), read_only=False)

con.execute("SELECT COUNT(*) AS rows FROM stg_registrations_clean").fetchdf()


,rows
0,128719


3) Create dim_date

In [2]:
con.execute("DROP TABLE IF EXISTS dim_date")

con.execute("""
CREATE TABLE dim_date AS
SELECT DISTINCT
  Report_date AS date,
  EXTRACT(year FROM Report_date) AS year,
  EXTRACT(month FROM Report_date) AS month,
  strftime(Report_date, '%Y-%m') AS year_month
FROM stg_registrations_clean
""")


In [3]:
con.execute("SELECT * FROM dim_date ORDER BY date LIMIT 10").fetchdf()


,date,year,month,year_month
0,2023-01-01,2023,1,2023-01
1,2024-01-01,2024,1,2024-01
2,2025-01-01,2025,1,2025-01


4) Create dim_state + identify the “17th state”

In [4]:
con.execute("DROP TABLE IF EXISTS dim_state")

con.execute("""
CREATE TABLE dim_state AS
SELECT DISTINCT
  state
FROM stg_registrations_clean
""")

con.execute("SELECT * FROM dim_state ORDER BY state").fetchdf()


,state
0,Baden-Württemberg
1,Bayern
2,Berlin
3,Brandenburg
4,Bremen
5,Hamburg
6,Hessen
7,Mecklenburg-Vorpommern
8,Niedersachsen
9,Nordrhein-Westfalen


5) Create dim_manufacturer

In [5]:
con.execute("DROP TABLE IF EXISTS dim_manufacturer")

con.execute("""
CREATE TABLE dim_manufacturer AS
SELECT DISTINCT
  manufacturer
FROM stg_registrations_clean
""")


6) Create dim_model (Trade name + Type key)

    We create a unique model identifier based on both fields.

In [6]:
con.execute("DROP TABLE IF EXISTS dim_model")

con.execute("""
CREATE TABLE dim_model AS
SELECT DISTINCT
  trade_name,
  type_key
FROM stg_registrations_clean
""")


7) Add surrogate keys to dimensions (very AE/DE)

   DuckDB doesn’t have sequences like Postgres, but we can generate stable keys using dense_rank().

In [7]:
con.execute("DROP TABLE IF EXISTS dim_date_keyed")
con.execute("""
CREATE TABLE dim_date_keyed AS
SELECT
  dense_rank() OVER (ORDER BY date) AS date_id,
  *
FROM dim_date
""")

con.execute("DROP TABLE IF EXISTS dim_state_keyed")
con.execute("""
CREATE TABLE dim_state_keyed AS
SELECT
  dense_rank() OVER (ORDER BY state) AS state_id,
  *
FROM dim_state
""")

con.execute("DROP TABLE IF EXISTS dim_manufacturer_keyed")
con.execute("""
CREATE TABLE dim_manufacturer_keyed AS
SELECT
  dense_rank() OVER (ORDER BY manufacturer) AS manufacturer_id,
  *
FROM dim_manufacturer
""")

con.execute("DROP TABLE IF EXISTS dim_model_keyed")
con.execute("""
CREATE TABLE dim_model_keyed AS
SELECT
  dense_rank() OVER (ORDER BY trade_name, type_key) AS model_id,
  *
FROM dim_model
""")


8) Create fact table fct_registrations

In [8]:
con.execute("DROP TABLE IF EXISTS fct_registrations")

con.execute("""
CREATE TABLE fct_registrations AS
SELECT
  d.date_id,
  s.state_id,
  m.manufacturer_id,
  mo.model_id,
  r.registrations_count,
  r.Object_Id
FROM stg_registrations_clean r
JOIN dim_date_keyed d
  ON r.Report_date = d.date
JOIN dim_state_keyed s
  ON r.state = s.state
JOIN dim_manufacturer_keyed m
  ON r.manufacturer = m.manufacturer
JOIN dim_model_keyed mo
  ON r.trade_name = mo.trade_name
 AND r.type_key = mo.type_key
""")


9) Validate the star schema

In [9]:
con.execute("""
SELECT
  (SELECT COUNT(*) FROM fct_registrations) AS fact_rows,
  (SELECT COUNT(*) FROM stg_registrations_clean) AS stg_rows
""").fetchdf()


,fact_rows,stg_rows
0,128430,128719


In [10]:
con.execute("""
SELECT
  COUNT(*) AS fact_rows,
  COUNT(DISTINCT date_id) AS dates,
  COUNT(DISTINCT state_id) AS states,
  COUNT(DISTINCT manufacturer_id) AS manufacturers,
  COUNT(DISTINCT model_id) AS models
FROM fct_registrations
""").fetchdf()


,fact_rows,dates,states,manufacturers,models
0,128430,3,17,83,2802


In [11]:
con.close()


Diagnistics:
1️⃣ Find rows that failed joins

In [12]:
con = duckdb.connect("../data/duckdb/motorcycle.db", read_only=False)

con.execute("""
SELECT
  COUNT(*) AS unmatched_rows
FROM stg_registrations_clean r
LEFT JOIN fct_registrations f
  ON r.Object_Id = f.Object_Id
WHERE f.Object_Id IS NULL
""").fetchdf()


,unmatched_rows
0,289


In [13]:
con.execute("""
SELECT
  r.Report_date,
  r.state,
  r.manufacturer,
  r.trade_name,
  r.type_key,
  r.registrations_count
FROM stg_registrations_clean r
LEFT JOIN fct_registrations f
  ON r.Object_Id = f.Object_Id
WHERE f.Object_Id IS NULL
LIMIT 30
""").fetchdf()


,Report_date,state,manufacturer,trade_name,type_key,registrations_count
0,2024-01-01,Schleswig-Holstein,BETAMOTOR (I),None,ABM,63
1,2024-01-01,Hamburg,BETAMOTOR (I),None,ABM,4
2,2024-01-01,Niedersachsen,BETAMOTOR (I),None,ABM,395
3,2024-01-01,Bremen,BETAMOTOR (I),None,ABM,8
4,2024-01-01,Nordrhein-Westfalen,BETAMOTOR (I),None,ABM,517
5,2024-01-01,Hessen,BETAMOTOR (I),None,ABM,275
6,2024-01-01,Rheinland-Pfalz,BETAMOTOR (I),None,ABM,164
7,2024-01-01,Baden-Württemberg,BETAMOTOR (I),None,ABM,684
8,2024-01-01,Bayern,BETAMOTOR (I),None,ABM,1193
9,2024-01-01,Saarland,BETAMOTOR (I),None,ABM,27


In [14]:
con.execute("""
SELECT
  SUM(CASE WHEN trade_name IS NULL OR TRIM(trade_name) = '' THEN 1 ELSE 0 END) AS missing_trade_name,
  SUM(CASE WHEN type_key IS NULL OR TRIM(type_key) = '' THEN 1 ELSE 0 END) AS missing_type_key,
  COUNT(*) AS unmatched_rows
FROM stg_registrations_clean r
LEFT JOIN fct_registrations f
  ON r.Object_Id = f.Object_Id
WHERE f.Object_Id IS NULL
""").fetchdf()


,missing_trade_name,missing_type_key,unmatched_rows
0,289.0,0.0,289


In [16]:
con.close()


“Some registrations were reported only at manufacturer–type level without a model name.
To preserve dimensional integrity, I excluded those rows from the model-level fact table and documented the data loss.”